<a href="https://colab.research.google.com/github/shubdas9902/AI-ML_projects/blob/main/VideoTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install torch  pyttsx3 transformers openvino youtube-dl ffmpeg-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.8 MB/s eta 0:00:00


In [3]:
!pip install git+https://github.com/openai/whisper.git




  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-3yg9onzg
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-3yg9onzg
  Resolved https://github.com/openai/whisper.git to commit 90db0de1896c23cbfaf0c58bc2d30665f709f170
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803583 sha256=d02558d6f03e244fe2f4de3a4394f71c682b85f34950c8e15f2031202abe767d
  Stored in directory: /tmp/pip-ephem-wheel-cache-cq0s1unj/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [50]:
!sudo apt-get install espeak-ng


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,526 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpcaudio0 amd64 1.1-6build2 [8,956 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 espeak-ng-data amd64 1.50+dfsg-10ubuntu0.1 [3,956 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libespeak-ng1 amd64 1.50+dfsg-10ubuntu0.1 [207 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 espeak-ng amd64 1.50+dfsg-1

In [45]:
def download_audio_from_video(video_url):
    print("Downloading video and extracting audio...")
    ydl_opts = {
        'format': 'bestaudio/best',  # Extract only the best audio
        'outtmpl': '/content/downloaded_audio.%(ext)s',  # Save audio with the appropriate extension
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

    # The file will be downloaded in .webm format by default
    audio_file = '/content/downloaded_audio.webm'  # Correct file extension
    print(f"Audio extracted: {audio_file}")

    # Convert the audio to a format that Whisper can process (e.g., WAV)
    converted_audio_file = '/content/downloaded_audio.wav'
    ffmpeg.input(audio_file).output(converted_audio_file).run()

    print(f"Audio converted to WAV: {converted_audio_file}")
    return converted_audio_file

In [27]:
def transcribe_audio_whisper(audio_file):
    import whisper
    model = whisper.load_model("base")  # You can use a different model size
    result = model.transcribe(audio_file)
    return result['text']


In [35]:
!pip install -U yt-dlp


In [38]:
import yt_dlp as youtube_dl


In [37]:
import torch
import numpy as np
import wave
import ffmpeg
import pyttsx3
from transformers import MarianMTModel, MarianTokenizer

import os
from google.colab import files

In [39]:
def translate_text(text, src_lang, tgt_lang):
    # Load the MarianMT model for translation
    model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}'
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)

    # Translate the text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)

    return translated_text

In [40]:
def text_to_speech_openvino(text, lang):
    # Initialize pyttsx3 TTS engine (OpenVINO™ doesn't directly support TTS, so we use pyttsx3 for now)
    engine = pyttsx3.init()

    # Set properties for TTS
    engine.setProperty('voice', lang)
    engine.setProperty('rate', 150)

    # Save speech as audio
    output_audio_file = '/content/output.mp3'
    engine.save_to_file(text, output_audio_file)
    engine.runAndWait()
    return output_audio_file

In [41]:
def overlay_audio_on_video(video_url, audio_file):
    # Download the video using youtube_dl
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': '/content/downloaded_video.%(ext)s',  # Save video with its original extension
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

    # Extract video and audio separately
    video_file = '/content/downloaded_video.mp4'
    audio_file_out = '/content/extracted_audio.mp3'

    # Extract audio from video using FFmpeg
    ffmpeg.input(video_file).output(audio_file_out).run()

    # Mute the original audio and overlay the new audio (translated speech)
    output_video_file = '/content/final_video_with_dubbing.mp4'
    ffmpeg.input(video_file).output(output_video_file, audio=audio_file).run()

    print(f"Final video with translated audio saved to {output_video_file}")
    return output_video_file

In [42]:
def process_video_for_translation_and_dubbing(video_url, src_lang, tgt_lang):
    # 1. Download and extract audio from YouTube video
    audio_file = download_audio_from_video(video_url)
    print(f"Audio extracted: {audio_file}")


    # Step 2: Transcribe audio to text using Whisper
    transcribed_text = transcribe_audio_whisper(audio_file)  # Using Whisper
    print(f"Transcribed Text: {transcribed_text}")

    # Step 3: Translate text dynamically
    translated_text = translate_text(transcribed_text, src_lang, tgt_lang)
    print(f"Translated Text: {translated_text}")

    # Step 4: Convert translated text to speech
    translated_audio_file = text_to_speech_openvino(translated_text, lang=tgt_lang)  # Using pyttsx3 for TTS

    # Step 5: Overlay translated speech on original video
    final_video = overlay_audio_on_video(video_url, translated_audio_file)

    # Allow the user to download the final video
    files.download(final_video)

    return final_video

In [67]:
video_url = "https://www.youtube.com/watch?v=c-e3xHi0onw"  # Replace with your YouTube video URL
src_lang = "en"  # Source language (English)
tgt_lang = "hi"  # Target language (Spanish)
final_video = process_video_for_translation_and_dubbing(video_url, src_lang, tgt_lang)

[youtube] Extracting URL: https://www.youtube.com/watch?v=c-e3xHi0onw
[youtube] c-e3xHi0onw: Downloading webpage
[youtube] c-e3xHi0onw: Downloading ios player API JSON
[youtube] c-e3xHi0onw: Downloading mweb player API JSON
[youtube] c-e3xHi0onw: Downloading m3u8 information
[info] c-e3xHi0onw: Downloading 1 format(s): 251
[download] /content/downloaded_audio.webm has already been downloaded
[download] 100% of    2.80MiB
Audio extracted: /content/downloaded_audio.wav


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

Transcribed Text:  Nai Bro там你這 command unf Horror or we still got the annual heard a normally not available youtube if you are China and if that we are have will better be came well the Amazon breathe has shared we have had gr�� ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔ ۔


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

...

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Translated Text: नीवुड या हम अभी भी एक नियमित रूप से उपलब्ध नहीं हैं... ... आप चीन हैं अगर और अगर हम बेहतर हैं की प्रतीक्षा की गई है की हम अच्छी तरह से PRIL PRICK Spe You Meol You MAPI © © © ©  ज़रा भी नहीं
[youtube] Extracting URL: https://www.youtube.com/watch?v=c-e3xHi0onw
[youtube] c-e3xHi0onw: Downloading webpage
[youtube] c-e3xHi0onw: Downloading ios player API JSON
[youtube] c-e3xHi0onw: Downloading mweb player API JSON
[youtube] c-e3xHi0onw: Downloading m3u8 information
[info] Testing format 616
[info] c-e3xHi0onw: Downloading 1 format(s): 616+251
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 51
[download] Destination: /content/downloaded_video.f616.mp4
[download] 100% of   81.29MiB in 00:00:06 at 12.06MiB/s                
[download] Destination: /content/downloaded_video.f251.webm
[download] 100% of    2.80MiB in 00:00:00 at 21.25MiB/s  
[Merger] Merging formats into "/content/downloaded_video.webm"
Deleting original file /content/downloaded_video.f616.mp

Error: ffmpeg error (see stderr output for detail)

In [65]:
import os

# Check if the file exists after conversion
audio_file = '/content/downloaded_audio.webm'
converted_audio_file = '/content/downloaded_audio.wav'

# Check if both files exist
print(f"WebM file exists: {os.path.exists(audio_file)}")
print(f"WAV file exists: {os.path.exists(converted_audio_file)}")



WebM file exists: True
WAV file exists: False


In [66]:
import subprocess

def convert_webm_to_wav(input_file, output_file):
    try:
        # Run FFmpeg conversion
        result = subprocess.run(
            ['ffmpeg', '-i', input_file, output_file],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            check=True
        )
        print(f"Conversion successful: {result.stdout.decode()}")
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg error: {e.stderr.decode()}")

# Convert the .webm file to .wav
convert_webm_to_wav('/content/downloaded_audio.webm', '/content/downloaded_audio.wav')


Conversion successful: 


In [52]:
!ffmpeg -i /content/downloaded_video.f616.mp4 -i /content/downloaded_audio.wav -c:v copy -c:a aac /content/output_video.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [53]:
!ls /content/


downloaded_audio.wav  downloaded_audio.webm  downloaded_video.webm  output.mp3	sample_data


In [54]:
!ffmpeg -i /content/downloaded_video.webm -i /content/downloaded_audio.wav -c:v libvpx -c:a libvorbis /content/final_video.webm


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [68]:
import subprocess

# Command to combine video and dubbed audio
command = [
    "ffmpeg", "-i", "downloaded_video.webm", "-i", "output.mp3",
    "-c:v", "copy", "-c:a", "aac", "-strict", "experimental", "final_video.mp4"
]

# Run the command
subprocess.run(command)


CompletedProcess(args=['ffmpeg', '-i', 'downloaded_video.webm', '-i', 'output.mp3', '-c:v', 'copy', '-c:a', 'aac', '-strict', 'experimental', 'final_video.mp4'], returncode=0)

In [58]:
import os

# Check if the final video exists
if os.path.exists("final_video.mp4"):
    print("Final video created successfully!")
else:
    print("Final video was not created.")


Final video created successfully!


In [63]:
from google.colab import files

# Download the final video file
files.download("final_video.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
!ffmpeg -i downloaded_video.webm -i output.mp3 -c:v copy -c:a aac -map 0:v:0 -map 1:a:0 -shortest final_video.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab